### Notebook deals with making a prelimnary code to analysis temporal wavefront error data.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

from astropy.io import fits
from matplotlib.colors import LogNorm, TwoSlopeNorm
import numpy as np
import scipy.io
import hcipy

import pastis.util as util
from pastis.config import CONFIG_PASTIS
from pastis.simulators.luvoir_imaging import LuvoirA_APLC
from pastis.simulators.scda_telescopes import HexRingAPLC

os.chdir('../ULTRA')
from config import CONFIG_ULTRA

In [ ]:
data_path = CONFIG_ULTRA.get('local_path', 'local_data_path')
analysis_path = CONFIG_ULTRA.get('local_path', 'local_analysis_path')

input_misalignments = np.genfromtxt(os.path.join(data_path,'INPUT_MISALIGNMENTS.txt'),  delimiter=';')
output_misalignments = np.genfromtxt(os.path.join(data_path,'RESIDUAL_MISALIGNMENTS.txt'),  delimiter=';')

In [ ]:
print("input_misalignment shape:", input_misalignments.shape)
print("output_misalignment shape:", output_misalignments.shape)

## For LUOVIR A 

### Unpacking data from .mat file 

In [ ]:
#sensitivities = scipy.io.loadmat(os.path.join(data_path, 'dWFE_sensitivities_LEC.mat'))

# seg1_hexike_coeffs = (sensitivities['hfit_all'][:, :, 0])
# seg2_hexike_coeffs = (sensitivities['hfit_all'][:, :, 1])   # segn = (mat['hfit_all'][:, :, n])

#np.savetxt("/Users/asahoo/Downloads/seg1.csv", seg1, delimiter=",")

#### Intantiate the LUVOIR A telescope simulator

In [ ]:
# optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
# sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')
# tel = LuvoirA_APLC(optics_input, 'small', sampling)

#### The following line takes ~5 minutes to execute in local machine.  

In [ ]:
# n_zernikes = 11      # Set the total number of hexike polynomials you want over the hexagonal segment.
# tel.create_segmented_mirror(n_zernikes) 

In [ ]:
# unaberrated_coro_psf, ref = tel.calc_psf(ref=True, display_intermediate=False, norm_one_photon=True)
# norm = np.max(ref)
# dh_intensity = (unaberrated_coro_psf / norm) * tel.dh_mask
# contrast_floor = np.mean(dh_intensity[np.where(tel.dh_mask != 0)])
# print(f'static contrast floor for luvoirA small APLC design: {contrast_floor}')

In [ ]:
# num_actuators = len(tel.sm.actuators) # ensure this to be equal to (total_segments) * (total_hexikes_per_segment)

In [ ]:
# (tel.sm.actuators).shape

In [ ]:
# sensitivities_table = sensitivities['hfit_all']
# luvoir_hexike_coeffs = []

# #removing the outer_ring
# for seg in range(0, 91):
#     seg_hexike_coeffs = sensitivities['hfit_all'][:, :, seg]
#     luvoir_hexike_coeffs.append(seg_hexike_coeffs)

In [ ]:
#sensitivities_table.shape
# seg1_hexike_coeffs = (sensitivities['hfit_all'][:, :, 0])

In [ ]:
# seg1_hexike_coeffs.shape
# luvoir_hexike_coeffs[0][0][0]

In [ ]:
# Note, Hexikes are set according to Noll Convention.

# tel.sm.flatten()
# for seg in range(0, 91):
#     for hexike in range(0, n_zernikes):
#         tel.sm.actuators[hexike + seg * n_zernikes] = (luvoir_hexike_coeffs[seg][2][hexike])*1e-12
        

# aberrated_coro_psf, efields_inter = tel.calc_psf(display_intermediate=False,return_intermediate='efield', norm_one_photon=True)

In [ ]:
# plt.figure()
# hcipy.imshow_field((efields_inter['seg_mirror']).phase, mask = tel.aperture, cmap='jet', vmin=-1e-4, vmax=0.0)
# plt.colorbar()
#plt.title('Segmented mirror phase)
#plt.savefig(os.path.join(analysis_path, 'luvoir_drz.png'))

## For SCDA 2hex Simulator

#### Instantiate the 2hex simulator.

In [ ]:
optics_dir = os.path.join(util.find_repo_location(), 'data', 'SCDA')
NUM_RINGS = 2
sampling = 4

tel2 = HexRingAPLC(optics_dir, NUM_RINGS, sampling)

unaberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=True, 
                                  return_intermediate='intensity',
                                  norm_one_photon=True)

norm = np.max(ref)
normalized_unaberrated_psf = unaberrated_psf / norm

unaberr_roi = normalized_unaberrated_psf * tel2.dh_mask
contrast_floor = np.mean(unaberr_roi[np.where(tel2.dh_mask != 0)])
print("contrast_floor:", contrast_floor)

In [ ]:
print(normalized_unaberrated_psf[5001], "\n")
print(unaberr_roi[5001])
print(tel2.wvln)

### Plot contrast as a function of the radius.

In [ ]:
grid =  hcipy.make_uniform_grid([115, 115], 115/2, center=115/2)

img_field1 = hcipy.Field(normalized_unaberrated_psf.shaped, grid)

plt.figure(figsize = (12, 5))

r1, profile1, _, _ = hcipy.radial_profile(img_field1, 1, statistic = 'mean')
plt.plot(r1, np.log10(profile1))


plt.show()

In [ ]:
# grid =  hcipy.make_uniform_grid([115, 115], 100/2, center=115/2)

# #img1 = np.reshape()

# img_field1 = hcipy.Field(normalized_unaberrated_psf, grid)
# img_field2 = hcipy.Field(unaberr_roi, grid)

# plt.figure(figsize = (12, 5))

# plt.subplot(1, 2, 1)
# r1, profile1, _, _ = hcipy.radial_profile(img_field1, 1, statistic = 'mean')
# # plt.xlim(55, 105)
# # plt.ylim(0, 1e-10)
# plt.plot(r1, np.log10(profile1))


# plt.subplot(1, 2, 2)
# r2, profile2, _, _ = hcipy.radial_profile(img_field2, 1, statistic = 'mean')
# plt.plot(r2, np.log10(profile2))
# #plt.ylim(0, 1e-10)
# #plt.xlim(55, 105)

# plt.show()

#### Load HWO sensitivities .mat file

In [ ]:
hwo_sensitivities = scipy.io.loadmat(os.path.join(data_path, 'HWO_sens_old.mat'))
hwo_table = hwo_sensitivities['HWO_sens']
first_element = hwo_table[0,0]
data_list = first_element.tolist()  # tuple of length 4

In [ ]:
hwo_hexike_coeffs = data_list[0]  # ndarray of shape (11, 19, 6)
mask = data_list[1]    # ndarray of shape (256, 256, 19, 6)
dopd = data_list[2]    # ndarray of shape (256, 256, 19, 6)
units = data_list[3]   # ndarray of shape (1,) where the only element is a string

#### Locate segment number using mask data

In [ ]:
seg = 15 # starts with 0 till 18
dof = 5 # starts with 0 till 5

plt.figure(figsize=(8, 3))
plt.subplot(1, 2, 1)
plt.title("mask")
plt.imshow(mask[:, :, seg, dof])
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("dopd")
plt.imshow(dopd[:, :, seg, dof], cmap='jet')
plt.colorbar()

#### Plot surface maps using the opd data only

In [ ]:
dopds = []

for dof in range(0, dopd.shape[3]):
    full_dopd = np.zeros((dopd.shape[1], dopd.shape[1]))
    for seg in range(0, dopd.shape[2]):
        opd_per_segment = dopd[:, :, seg, dof]
        full_dopd = opd_per_segment + full_dopd
        
    dopds.append(full_dopd)
    
titles = np.array(["X nm RMS/um", "Y nm RMS/um", "Z nm RMS/um", 
                  "Rx nm RMS/urad", "Ry nm RMS/urad", "Rz nm RMS/urad" ])
        
plt.figure(figsize = (10, 5))
for dof in range(0, len(dopds)):
    if dof!=2:
        plt.subplot(2, 3, dof+1)
        plt.title(titles[dof], fontweight = 'bold', fontsize= 7)
        plt.imshow(dopds[dof], cmap='jet')
        plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=7)
        cbar = plt.colorbar()
        cbar.ax.tick_params(labelsize=10)

plt.subplot(2, 3, 3)
plt.title("Z nm RMS/um", fontweight = 'bold', fontsize=10)
plt.imshow(dopds[2], cmap='jet', vmin=-2000, vmax= -1966)
plt.tick_params(top=False, bottom=True, left=True, right=False, labelleft=True, labelbottom=True, labelsize=7)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=7)

plt.tight_layout()
#plt.savefig(os.path.join(analysis_path, 'hwo_dopds_PTT.png'))

In [ ]:
#do not understand why they sent me multiple masks, 
#I plotted all the masks per dof, subtracted two masks to check if there is some mismatch between dofs, 
#they are identical to 1e-30 scientific precision.

masks = []

for dof in range(0, mask.shape[3]):
    full_mask = np.zeros((mask.shape[1], mask.shape[1]))
    for seg in range(0, mask.shape[2]):
        mask_per_segment = mask[:, :, seg, dof]
        full_mask = mask_per_segment + full_mask
        
    masks.append(full_mask)
    
titles = np.array(["mask (X nm RMS/um)", "mask (Y nm RMS/um)", "mask (Z nm RMS/um)", 
                  "mask (Rx nm RMS/urad)", "mask (Ry nm RMS/urad)", "mask (Rz nm RMS/urad)"])
    
plt.figure(figsize = (14, 7))
for dof in range(0, len(dopds)):
    plt.subplot(2, 3, dof+1)
    plt.title(titles[dof], fontweight = 'bold')
    plt.imshow(masks[dof], cmap='jet')
    plt.colorbar()

In [ ]:
dopds = np.array(dopds)
masks = np.array(masks)

titles = np.array(["X nm RMS/um", "Y nm RMS/um", "Z nm RMS/um", 
                  "Rx nm RMS/urad", "Ry nm RMS/urad", "Rz nm RMS/urad" ])

for dof in range(0, len(dopds)):
    rms = np.sqrt((np.mean(dopds[dof][np.where(masks[dof]!= 0)]**2)))
    print(titles[dof],"----", rms)

### Map HWO segment number to the SCDA segment number:

In [ ]:
hwo_to_scda_L = {"1": 4, "2": 3, "3": 2, "4":7, "5":6, "6":5, "7":12, "8":11, "9":10, "10":9,
               "11": 8, "12":19, "13":18, "14":17, "15":16, "16":15, "17":14, "18":13, "19":1}

In [ ]:
hwo_to_scda = {"1": 4, "2": 5, "3": 6, "4":7, "5":2, "6":3, "7":12, "8":13, "9":14, "10":15,
               "11": 16, "12":17, "13":18, "14":19, "15":8, "16":9, "17":10, "18":11, "19":1}

hwo_scda_segments = np.array([4, 5, 6, 7, 2, 3, 12, 13, 14, 15, 16, 17, 18, 19, 8, 9, 10, 11, 1])

In [ ]:
seg1_table = hwo_hexike_coeffs[:, 12, :].T

plt.figure(figsize = (15, 5))
plt.subplot(1, 2, 1)
plt.imshow((seg1_table))
plt.ylabel('DOF freedom')
plt.xlabel('Zernike coefficients')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(np.arange(1, 12, 1), seg1_table[0, :], label = 'for x')
# plt.plot(np.arange(1, 12, 1), seg1_table[1, :], label = 'for y')
# plt.plot(np.arange(1, 12, 1), seg1_table[2, :], label = 'for z')
# plt.plot(np.arange(1, 12, 1), seg1_table[3, :], label = 'for rx')
# plt.plot(np.arange(1, 12, 1), seg1_table[4, :], label = 'for ry')
# plt.plot(np.arange(1, 12, 1), seg1_table[5, :], label = 'for rz')
plt.ylabel('coefficients')
plt.xlabel('Zernike mode')
#plt.ylim(-1, 0)
plt.colorbar()
plt.legend()



In [ ]:
print("Piston", "Tip", "Tilt")
print((seg1_table[0, 0:3]))
print((seg1_table[1, 0:3]))
print((seg1_table[2, 0:3]))
print((seg1_table[3, 0:3]))
print((seg1_table[4, 0:3]))
print((seg1_table[5, 0:3]))

In [ ]:
segs_tables = []
for seg in range(0, 19):
    seg_table = hwo_hexike_coeffs[:, seg, :].T
    segs_tables.append(seg_table)

In [ ]:
n_zernikes = 11      # Set the total number of hexike polynomials you want over the hexagonal segment.
tel2.create_segmented_mirror(n_zernikes)

In [ ]:
psf_2d = np.array(np.reshape(normalized_unaberrated_psf, (115, 115)))

#fits.writeto((os.path.join(analysis_path, 'ideal_psf.fits')), np.reshape(ref, (115, 115)))
# ideal_psf = np.reshape(ref, [115,115])

####tel2.sampling pixels = l/d

# ind_max = np.unravel_index(np.argmax(np.abs(ideal_psf), axis=None), np.abs(ideal_psf).shape)
# for i in range(0,135,20):
#     print(i, (i-ind_max[0])*(1/tel2.sampling)) 
    
x_pixels = [0, 20 ,40, 60, 80, 100]
x_ld = [r"-14.25$\lambda/D$",r"-9.25$\lambda/D$", r"-4.25$\lambda/D$", r"0.75$\lambda/D$",
        r"5.75$\lambda/D$", r"10.75$\lambda/D$"]

plt.figure(figsize = (20, 5))

plt.subplot(1, 3, 1)
plt.title("Mirror surface")
hcipy.imshow_field(tel2.sm.surface*1e9, mask=tel2.aperture, cmap='RdBu')
cbar = plt.colorbar()
plt.xlabel("in m")
plt.ylabel("in m")
cbar.set_label("in nm", loc='center')

plt.subplot(1, 3, 2)
plt.imshow(np.log10(psf_2d), origin='lower', cmap= 'inferno')
plt.title("Coronagraphic PSF incase of \n Hexike-coefficients set to Zero.")
plt.xticks(x_pixels, x_ld, fontsize = 8)
plt.yticks(x_pixels, x_ld, fontsize = 8)
plt.xlabel(r"Angular Separation $(\lambda/D)$")
plt.ylabel(r"Angular Separation $(\lambda/D)$")
plt.colorbar()


plt.subplot(1, 3, 3)
#plt.plot(np.arange(0, 115, 1), np.log10(psf_2d[:, 56]), label='x')
plt.title("Intensity vs Radius")
plt.plot(np.arange(0, 115, 1), np.log10(psf_2d[56, :]))
plt.xlabel(r"Angular Separation $(\lambda/D)$")
plt.ylabel("Normalized Irradiance (log scale)")
plt.xticks(x_pixels, x_ld, fontsize = 8)
plt.tight_layout()
#plt.show()
#plt.savefig(os.path.join(analysis_path, 'zero_dm.png'))

In [ ]:
np.sqrt((tel2.sm.surface).shape)

In [ ]:
# tel2.sm.flatten()
# fits.writeto(os.path.join(analysis_path, 'pupil2hex_scda.fits'), (tel2.sm.surface).shaped)

In [ ]:
plt.figure(figsize = (14, 10))
plt.title("Zernike Coefficients ordering in PASTIS tel simulator")
for z in range(0, 11):
    tel2.sm.flatten()
    tel2.sm.actuators[z] = 1e-9

    plt.subplot(3, 4, int(z+1))
    plt.title(f"Z{z}", fontweight = 'bold')
    hcipy.imshow_field(tel2.sm.surface, cmap='jet')
    cbar = plt.colorbar()
    cbar.set_label("in m", loc='center')

In [ ]:
tel2.sm.flatten()

# multiply by 1/2 ?
tel2_surfaces = []
for dof in range(0, 6):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        for hexike in range(0, n_zernikes):
            scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * segs_tables[hwo_seg][dof][hexike] * 1e-9
    
    tel2_surfaces.append(tel2.sm.surface)
                      

titles = np.array(["X nm RMS/um", "Y nm RMS/um", "Z nm RMS/um", 
                  "Rx nm RMS/urad", "Ry nm RMS/urad", "Rz nm RMS/urad" ])
        
plt.figure(figsize = (14, 7))
plt.suptitle("Without sorting or changing sign of any Zernike mode", fontweight = 'bold', fontsize=20)
for dof in range(0, 6):
    if dof!=2:
        plt.subplot(2, 3, dof+1)
        plt.title(titles[dof], fontweight = 'bold')
        hcipy.imshow_field(tel2_surfaces[dof]*1e9, cmap='jet')
        cbar = plt.colorbar()
        cbar.set_label("in nm", loc='center')

plt.subplot(2, 3, 3)
plt.title("Z nm RMS/um", fontweight = 'bold')
hcipy.imshow_field(tel2_surfaces[2]*1e9, cmap='jet', vmin = -2000*1*0.5,  vmax= -1960*1*0.5)
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')  

plt.savefig(os.path.join(analysis_path, 'hwo_PTT.png'))

# aberrated_coro_psf2, efields_inter2 = tel2.calc_psf(display_intermediate=True, 
#                                                     return_intermediate='efield', 
#                                                     norm_one_photon=True)


#### Sorting Zernike coefficients

In [ ]:
tel2.sm.flatten()

tel2_surfaces_sorted = []
tel2_actuators = []
for dof in range(0, 6):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg): 
        for hexike in range(0, n_zernikes):
            scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
            if hexike == 0:
                hexn = 0
                sign = 2
            elif hexike == 1:
                hexn = 1
                sign = 1
            elif hexike == 2:
                hexn = 2
                sign = -1
            elif hexike == 3:
                hexn = 3
                sign = 1
            elif hexike == 4:
                hexn = 5
                sign = -1
            elif hexike == 5:
                hexn = 4
                sign = 1
            elif hexike == 6:
                hexn = 7
                sign = 1
            elif hexike == 7:
                hexn = 6
                sign = 1
            elif hexike == 8:
                hexn = 10
                sign = 1
            elif hexike == 9:
                hexn = 9
                sign = 1
            elif hexike == 10:
                hexn = 8
                sign = 1
            else:
                hexn = hexike
                sign = 1
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * 1 * sign * segs_tables[hwo_seg][dof][hexn] * 1e-9 
            
    tel2_surfaces_sorted.append(tel2.sm.surface)
    tel2_actuators.append(tel2.sm.actuators)

In [ ]:
titles = np.array(["X nm RMS/um", "Y nm RMS/um", "Z nm RMS/um", 
                  "Rx nm RMS/urad", "Ry nm RMS/urad", "Rz nm RMS/urad" ])
        
plt.figure(figsize = (14, 9))
#plt.suptitle("After sorting and flipping sign of some Zernike mode", fontweight = 'bold', fontsize=20)
for dof in range(0, 6):
    if dof!=2:
        rms_scda = np.sqrt(np.mean((tel2_surfaces_sorted[dof][np.where(tel2.aperture!= 0)])**2)) * 1e9
        plt.subplot(2, 3, dof+1)
        plt.title(titles[dof], fontweight = 'bold')
        hcipy.imshow_field(tel2_surfaces_sorted[dof]*1e9, cmap='jet')
        plt.xlabel(f'RMS: {rms_scda:.2f} nm')
        cbar = plt.colorbar()
        cbar.set_label("in nm", loc='center')

plt.subplot(2, 3, 3)
rms_scda = np.sqrt(np.mean((tel2_surfaces_sorted[2][np.where(tel2.aperture!= 0)])**2)) * 1e9
plt.title("Z nm RMS/um", fontweight = 'bold')
hcipy.imshow_field(tel2_surfaces_sorted[2]*1e9, cmap='jet', vmin = -2000 * 0.5 *2 ,  vmax= -1960* 0.5 *2)
plt.xlabel(f"RMS: {rms_scda:.2f} nm")
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')
#plt.savefig(os.path.join(analysis_path, 'hwo_PTT_sorted.png'))

### Propagate opds through the simulator. 

In [ ]:
dx0 = input_misalignments[:, 0] * 1e12 # in units of pm. 
dx0 = np.delete(dx0, -1)
print(dx0, "\n")

dx1 = input_misalignments[:, 6] * 1e12
dx1 = np.delete(dx1, -1)

print(dx1, "\n")


dx2 = input_misalignments[:, 12] * 1e12
dx2 = np.delete(dx2, -1)

print(dx2, "\n")

dx3 = input_misalignments[:, 18] * 1e12
dx3 = np.delete(dx3, -1)

print(dx3, "\n")

dx400 = input_misalignments[:, 400*6] * 1e12
dx400 = np.delete(dx400, -1)

print(dx400, "\n")

In [ ]:
# dx0[3] = 145
# print(dx0)

In [ ]:
tel2.sm.flatten()

for hwo_seg in range(0, tel2.nseg):
    scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
    for hexike in range(0, n_zernikes):
        #print(hwo_seg, scda_seg,  hexike + scda_seg * n_zernikes,)
        tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * dx1[hwo_seg] * 1e-6

# #tel2.sm.flatten()

# plt.figure()
# hcipy.imshow_field(tel2.sm.surface*1e9, cmap='jet')
# cbar = plt.colorbar()
# cbar.set_label("in nm", loc='center')
# plt.show()

aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, 
                                  return_intermediate='intensity',
                                  norm_one_photon=True)

normalized_aberrated_psf = aberrated_psf / norm

aberr_roi = normalized_aberrated_psf * tel2.dh_mask
aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
print("contrast_floor:", aber_contrast_floor)

fpm_mask = np.zeros(len(intermediates['after_fpm']))

for i in range(0, len(intermediates['after_fpm'])):
    if intermediates['after_fpm'][i] == 0.0:
        fpm_mask[i] = 0
    else:
        fpm_mask[i] = 1

plt.figure(figsize = (13, 7))

# Mirror surface
plt.subplot(2, 3, 1)
plt.title("Surface")
hcipy.imshow_field((tel2.sm.surface*1e9), mask=tel2.aperture, cmap='inferno')
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')
plt.xlabel("in m")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)

# Entrance pupil
plt.subplot(2, 3, 2)
plt.title("Apodizer plane")
hcipy.imshow_field(tel2.sm.surface*1e9, mask= tel2.apodizer, cmap='inferno')
plt.xlabel("in m")
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')

plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)

# before FPM
plt.subplot(2, 3, 3)
plt.title("Before FPM")
hcipy.imshow_field(intermediates['before_fpm'], 
                   norm=LogNorm(vmin=1e-8, vmax=1e-1), cmap='inferno')
plt.xlabel(r"in $\lambda/D$")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()

# after FPM, note: calc_psf returns int_after_fpm in log scale.
plt.subplot(2, 3, 4)
plt.title("After FPM")
plt.xlabel(r"in $\lambda/D$")
hcipy.imshow_field(10**(intermediates['after_fpm']), 
                   norm=LogNorm(vmin=1e-8, vmax=1e-1),mask = fpm_mask, cmap= 'inferno')
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()


# after Lyot Stop
plt.subplot(2, 3, 5)
plt.title("After Lyot stop")
hcipy.imshow_field(intermediates['after_lyot'], mask=tel2.lyotstop, norm=LogNorm(vmin=1e-3, vmax=1),
                   cmap='inferno')
plt.xlabel("in m")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()

# final PSF
plt.subplot(2, 3, 6)
plt.title("Coronagraphic PSF")
hcipy.imshow_field(aberrated_psf, norm=LogNorm(vmin=1e-14, vmax=1e-3), cmap='inferno')
plt.xlabel(r"in $\lambda/D$")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()

plt.tight_layout()
plt.savefig(os.path.join(analysis_path, 'dx1_open_loop.png'))

In [ ]:
tel2.sm.flatten()
dx_times = []
for time in range(0, 401):
    dx = output_misalignments[:, time*6] * 1e12 # in units of pm. 
    dx = np.delete(dx, -1)
    dx_times.append(dx)

contrasts_dx = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * dx_times[time][hwo_seg] * 1e-6

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_dx.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
plt.figure()
plt.plot(np.arange(0, 401, 1), contrasts_dx, label='dx DOF')
plt.axhline(y=4.171337358217274e-11, color='r', linestyle='--', label='contrast_floor')
plt.ylabel("Mean DH Contrast")
plt.xlabel("Time (in s)")
plt.legend()

In [ ]:
tel2.sm.flatten()
dy_times = []
for time in range(0, 401):
    dy = output_misalignments[:, time*6 + 1] * 1e12 # in units of pm. 
    dy = np.delete(dy, -1)
    dy_times.append(dy)

contrasts_dy = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * dy_times[time][hwo_seg] * 1e-6

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_dy.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
len(contrasts_dy)

In [ ]:
plt.figure()
plt.plot(np.arange(0, 401, 1), contrasts_dx, label='dx DOF')
plt.plot(np.arange(0, 401, 1), contrasts_dy, label='dy DOF')
plt.axhline(y=4.171337358217274e-11, color='r', linestyle='--', label='contrast_floor')
plt.ylabel("Mean DH Contrast")
plt.xlabel("Time (in s)")
plt.legend()

In [ ]:
tel2.sm.flatten()
dz_times = []
for time in range(0, 401):
    dz = output_misalignments[:, time*6 + 2] * 1e12 # in units of pm. 
    dz = np.delete(dz, -1)
    dz_times.append(dz)

contrasts_dz = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * dz_times[time][hwo_seg] * 1e-6

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_dz.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
tel2.sm.flatten()
rdx_times = []
for time in range(0, 401):
    rdx = output_misalignments[:, time*6 + 3] * 1e12 # in units of pm. 
    rdx = np.delete(rdx, -1)
    rdx_times.append(rdx)

contrasts_rdx = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * rdx_times[time][hwo_seg] * 1e-6

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_rdx.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
tel2.sm.flatten()
rdy_times = []
for time in range(0, 401):
    rdy = output_misalignments[:, time*6 + 4] * 1e12 # in units of pm. 
    rdy = np.delete(rdy, -1)
    rdy_times.append(rdy)

contrasts_rdy = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * rdy_times[time][hwo_seg] * 1e-6

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_rdy.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
tel2.sm.flatten()
rdz_times = []
for time in range(0, 401):
    rdz = output_misalignments[:, time*6 + 5] * 1e12 # in units of pm. 
    rdz = np.delete(rdz, -1)
    rdz_times.append(rdz)

contrasts_rdz = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * rdz_times[time][hwo_seg] * 1e-6

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_rdz.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
plt.figure(figsize=(20, 10))
times = np.arange(0, 401, 1)
plt.plot(times, contrasts_dx, marker='s', markersize=4, label='dx DOF')
plt.plot(times, contrasts_dy, marker='P', markersize=4, label='dy DOF')
plt.plot(times, contrasts_dz, marker='*', markersize=4, label='dz DOF')
plt.plot(times, contrasts_rdx, marker='X', markersize=4, label='rdx DOF')
plt.plot(times, contrasts_rdy, marker='v', markersize=4, label='rdy DOF')
plt.plot(times, contrasts_rdz, marker='^', markersize=4, label='rdz DOF')
plt.axhline(y=4.171337358217274e-11, color='r', linestyle='--', label='contrast_floor')
plt.ylabel("Mean DH Contrast")
plt.ylim(4.171e-11, 4.172e-11)
plt.xlabel("Time (in s)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(analysis_path, 'close_loop.png'))

In [ ]:
np.savetxt(os.path.join(analysis_path, 'contrast_dx.csv'), contrasts_dx, delimiter=',')
np.savetxt(os.path.join(analysis_path, 'contrast_dy.csv'), contrasts_dy, delimiter=',')
np.savetxt(os.path.join(analysis_path, 'contrast_dz.csv'), contrasts_dz, delimiter=',')
np.savetxt(os.path.join(analysis_path, 'contrast_rdx.csv'), contrasts_rdx, delimiter=',')
np.savetxt(os.path.join(analysis_path, 'contrast_rdy.csv'), contrasts_rdy, delimiter=',')
np.savetxt(os.path.join(analysis_path, 'contrast_rdz.csv'), contrasts_rdz, delimiter=',')

In [ ]:
dx_times[0]

### Propagating the calibration opd (opds due to 1 um or 1 urad displacement)

In [ ]:
tel2.sm.flatten()

tel2.sm.actuators = 0.5 * tel2_actuators[5]
# vmin=-2000*0.5, vmax= -1960*0.5
# plt.figure()
# plt.title("Surface")
# hcipy.imshow_field((tel2.sm.surface*1e9), cmap='jet')
# cbar = plt.colorbar()
# cbar.set_label("in nm", loc='center')
# plt.show()

aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, 
                                  return_intermediate='intensity',
                                  norm_one_photon=True)

normalized_aberrated_psf = aberrated_psf / norm

aberr_roi = normalized_aberrated_psf * tel2.dh_mask
aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
print("contrast_floor:", aber_contrast_floor)



fpm_mask = np.zeros(len(intermediates['after_fpm']))

for i in range(0, len(intermediates['after_fpm'])):
    if intermediates['after_fpm'][i] == 0.0:
        fpm_mask[i] = 0
    else:
        fpm_mask[i] = 1

plt.figure(figsize = (13, 7))

# Mirror surface
plt.subplot(2, 3, 1)
plt.title("Surface")
hcipy.imshow_field((tel2.sm.surface*1e9), mask=tel2.aperture, cmap='inferno')
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')
plt.xlabel("in m")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)

# Entrance pupil
plt.subplot(2, 3, 2)
plt.title("Apodizer plane")
hcipy.imshow_field(tel2.sm.surface*1e9, mask= tel2.apodizer, cmap='inferno')
plt.xlabel("in m")
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')

plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)

# before FPM
plt.subplot(2, 3, 3)
plt.title("Before FPM")
hcipy.imshow_field(intermediates['before_fpm'], 
                   norm=LogNorm(vmin=1e-8, vmax=1e-1), cmap='inferno')
plt.xlabel(r"in $\lambda/D$")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()

# after FPM, note: calc_psf returns int_after_fpm in log scale.
plt.subplot(2, 3, 4)
plt.title("After FPM")
plt.xlabel(r"in $\lambda/D$")
hcipy.imshow_field(10**(intermediates['after_fpm']), 
                   norm=LogNorm(vmin=1e-8, vmax=1e-1),mask = fpm_mask, cmap= 'inferno')
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()


# after Lyot Stop
plt.subplot(2, 3, 5)
plt.title("After Lyot stop")
hcipy.imshow_field(intermediates['after_lyot'], mask=tel2.lyotstop, norm=LogNorm(vmin=1e-3, vmax=1),
                   cmap='inferno')
plt.xlabel("in m")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()

# final PSF
plt.subplot(2, 3, 6)
plt.title("Coronagraphic PSF")
hcipy.imshow_field(aberrated_psf, norm=LogNorm(vmin=1e-14, vmax=1e-3), cmap='inferno')
plt.xlabel(r"in $\lambda/D$")
plt.tick_params(bottom=True, left=True, labelleft=True, labelbottom=True)
plt.colorbar()

plt.tight_layout()
plt.savefig(os.path.join(analysis_path, 'rdz_simulator.png'))

In [ ]:
tel2.lyotstop?

### Propagate all DOFs at a time

In [ ]:
tel2.sm.flatten()
dx_times = []
for time in range(0, 401):
    dx = output_misalignments[:, time*6] * 1e12 # in units of pm. 
    dx = np.delete(dx, -1)
    dx_times.append(dx)

dy_times = []
for time in range(0, 401):
    dy = output_misalignments[:, time*6 + 1] * 1e12 # in units of pm. 
    dy = np.delete(dy, -1)
    dy_times.append(dy)
    
dz_times = []
for time in range(0, 401):
    dz = output_misalignments[:, time*6 + 2] * 1e12 # in units of pm. 
    dz = np.delete(dz, -1)
    dz_times.append(dz) 
    
rdx_times = []
for time in range(0, 401):
    rdx = output_misalignments[:, time*6 + 3] * 1e12 # in units of pm. 
    rdx = np.delete(rdx, -1)
    rdx_times.append(rdx)
    
    
rdy_times = []
for time in range(0, 401):
    rdy = output_misalignments[:, time*6 + 4] * 1e12 # in units of pm. 
    rdy = np.delete(rdy, -1)
    rdy_times.append(rdy)
    
rdz_times = []
for time in range(0, 401):
    rdz = output_misalignments[:, time*6 + 5] * 1e12 # in units of pm. 
    rdz = np.delete(rdz, -1)
    rdz_times.append(rdz)
    

contrasts_dof = []
for time in range(0, 401):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg):
        scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
        for hexike in range(0, n_zernikes):
            dox = 0.5 * tel2_actuators[0][hexike + scda_seg * n_zernikes] * dx_times[time][hwo_seg] * 1e-6
            doy = 0.5 * tel2_actuators[1][hexike + scda_seg * n_zernikes] * dy_times[time][hwo_seg] * 1e-6
            doz = 0.5 * tel2_actuators[2][hexike + scda_seg * n_zernikes] * dz_times[time][hwo_seg] * 1e-6
            rdx = 0.5 * tel2_actuators[3][hexike + scda_seg * n_zernikes] * rdx_times[time][hwo_seg] * 1e-6
            rdy = 0.5 * tel2_actuators[4][hexike + scda_seg * n_zernikes] * rdy_times[time][hwo_seg] * 1e-6
            rdz = 0.5 * tel2_actuators[5][hexike + scda_seg * n_zernikes] * rdz_times[time][hwo_seg] * 1e-6
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] =  dox + doy + doz + rdx + rdy + rdz

    aberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, return_intermediate='intensity',norm_one_photon=True)
    normalized_aberrated_psf = aberrated_psf / norm
    aberr_roi = normalized_aberrated_psf * tel2.dh_mask
    aber_contrast_floor = np.mean(aberr_roi[np.where(tel2.dh_mask != 0)])
    contrasts_dof.append(aber_contrast_floor)
    print("contrast_floor:", aber_contrast_floor)

In [ ]:
contrasts_dx = np.genfromtxt(os.path.join(analysis_path, 'contrast_dx.csv'), delimiter=',')
contrasts_dy = np.genfromtxt(os.path.join(analysis_path, 'contrast_dy.csv'), delimiter=',')
contrasts_dz = np.genfromtxt(os.path.join(analysis_path, 'contrast_dz.csv'), delimiter=',')
contrasts_rdx = np.genfromtxt(os.path.join(analysis_path, 'contrast_rdx.csv'), delimiter=',')
contrasts_rdy = np.genfromtxt(os.path.join(analysis_path, 'contrast_rdy.csv'), delimiter=',')
contrasts_rdz = np.genfromtxt(os.path.join(analysis_path, 'contrast_rdz.csv'), delimiter=',')

In [ ]:
plt.figure(figsize=(20, 10))
times = np.arange(0, 401, 1)
plt.plot(times, contrasts_dof, marker='s', markersize=8, label='total')
plt.plot(times, contrasts_dx, marker='s', markersize=4)
plt.plot(times, contrasts_dy, marker='P', markersize=4, label='dy DOF')
plt.plot(times, contrasts_dz, marker='*', markersize=4, label='dz DOF')
plt.plot(times, contrasts_rdx, marker='X', markersize=4, label='rdx DOF')
plt.plot(times, contrasts_rdy, marker='v', markersize=4, label='rdy DOF')
plt.plot(times, contrasts_rdz, marker='^', markersize=4, label='rdz DOF')
plt.axhline(y=4.171337358217274e-11, color='r', linestyle='--', label='contrast_floor')
plt.ylabel("Mean DH Contrast")
plt.xlabel("Time (in s)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(analysis_path, 'total_close_loop.png'))

#### Remove PTT from dOPDs

In [ ]:
tel2.sm.flatten()
tel2_surfaces_withoutPTT = []
for dof in range(0, 6):
    tel2.sm.flatten()
    for hwo_seg in range(0, tel2.nseg): 
        for hexike in range(3, n_zernikes):
            scda_seg = hwo_to_scda[str(hwo_seg + 1)] - 1
            if hexike == 0:
                hexn = 0
                sign = 2
            elif hexike == 1:
                hexn = 1
                sign = 1
            elif hexike == 2:
                hexn = 2
                sign = -1
            elif hexike == 4:
                hexn = 5
                sign = -1
            elif hexike == 5:
                hexn = 4
                sign = 1
            elif hexike == 6:
                hexn = 7
                sign = 1
            elif hexike == 7:
                hexn = 6
                sign = 1
            elif hexike == 8:
                hexn = 10
                sign = 1
            elif hexike == 9:
                hexn = 9
                sign = 1
            elif hexike == 10:
                hexn = 8
                sign = 1
            else:
                hexn = hexike
            tel2.sm.actuators[hexike + scda_seg * n_zernikes] = 0.5 * sign * segs_tables[hwo_seg][dof][hexn] * 1e-9
    
    tel2_surfaces_withoutPTT.append(tel2.sm.surface)

In [ ]:
titles = np.array(["X nm RMS/um", "Y nm RMS/um", "Z nm RMS/um", 
                  "Rx nm RMS/urad", "Ry nm RMS/urad", "Rz nm RMS/urad" ])
        
plt.figure(figsize = (14, 9))
for dof in range(0, 6):
    if dof!=2:
        rms_scda = np.sqrt(np.mean((tel2_surfaces_withoutPTT[dof][np.where(tel2.aperture!= 0)])**2)) * 1e9
        plt.subplot(2, 3, dof+1)
        plt.title(titles[dof], fontweight = 'bold')
        hcipy.imshow_field(tel2_surfaces_withoutPTT[dof]*1e9, cmap='jet')
        plt.xlabel(f'RMS: {rms_scda:.5f} nm')
        cbar = plt.colorbar()
        cbar.set_label("in nm", loc='center')

plt.subplot(2, 3, 3)
rms_scda = np.sqrt(np.mean((tel2_surfaces_withoutPTT[2][np.where(tel2.aperture!= 0)])**2)) * 1e9
plt.title("Z nm RMS/um", fontweight = 'bold')
hcipy.imshow_field(tel2_surfaces_withoutPTT[2]*1e9, cmap='jet')
plt.xlabel(f"RMS: {rms_scda:.5f} nm")
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')
#plt.savefig(os.path.join(analysis_path, 'hwo_without_PTT_sorted.png'))

#### Calculate the percentage off from dopds' value

In [ ]:
print("With PTT")
print("For X dof:", round(np.abs((18.31 - 18.5)/18.5 * 100), 2),"%")
print("For Y dof:", round(np.abs((18.20 - 18.36)/18.36 * 100), 2),"%")
#print("For Z dof:", round(np.abs((18.31 - 18.5)/18.5 * 100), 2),"%")
print("For Rx dof:", round(np.abs((812.18 - 818.9)/819.9 * 100), 2),"%")
print("For Ry dof:", round(np.abs((803.33 - 812.1)/812.1 * 100), 2),"%")
print("For Rz dof:", round(np.abs((94.08 - 95.13)/95.13 * 100), 2),"%", "\n")


print("without PTT")
print("For X dof:", round(np.abs((0.04025 - 0.03382)/0.03382 * 100), 2),"%")
print("For Y dof:", round(np.abs((0.04398 - 0.03907)/0.03907 * 100), 2),"%")
print("For Z dof:", round(np.abs((0.17940 - 0.1993)/0.1993 * 100), 2),"%")
print("For Rx dof:", round(np.abs((0.88674 - 0.7349)/0.7349 * 100), 2),"%")
print("For Ry dof:", round(np.abs((0.85748 - 0.7118)/0.7118 * 100), 2),"%")
print("For Rz dof:", round(np.abs((0.23794 - 0.1977)/0.1977 * 100), 2),"%", "\n")

In [ ]:
#A = np.array([[1, 8, 9, 4], [2, 6, 7, 0], [1, 3, 8, 9], [0, 0, 4,1]])
# A = np.array([[1.6, 8, 9, 4], [2, 6, 7, 0], [1, 3, 8, 9]])
# print(A)

# A[:,[0,3]] = A[:,[3,0]]
# print(A)

# seg1_table[:, [5, 4]] = seg1_table[:, [4, 5]] # swapping astigmatism
# seg1_table[:, [7, 6]] = seg1_table[:, [6, 7]] # swapping coma

#### Plot the hexmaps from ball.
 

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

X = np.linspace(-10, 10, 1000)
Y = np.linspace(-10, 10, 1000)

th, R = cart2pol(X, -Y)

In [ ]:
z1 = np.ones(X.shape) 
z2 = np.sqrt(24/5)*X
z3 = np.sqrt(24/5)*Y
z4 = np.sqrt(720/43)*(R**2 - 5/12)
z5 = np.sqrt(60/7)*(R**2)*np.cos(2*th)
z6 = np.sqrt(60/7)*(R**2)*np.sin(2*th)
z7 = np.sqrt(84000/737)*(R**2 - 14/25)*R*np.cos(th)
z8 = np.sqrt(84000/737)*(R**2 - 14/25)*R*np.sin(th)
z9 = np.sqrt(1517040/4987)*(R**4 - 257/301*(R**2) + 737/6020)
z10 = np.sqrt(1120/103)*(R**3)*(np.cos(3*th))
z11 = np.sqrt(160/9)*(R**3)*(np.cos(3*th))

plt.figure()
plt.subplot(3, 4, 1)
plt.plot(X, z1)

plt.subplot(3, 4, 2)
plt.plot(X, z2)

plt.subplot(3, 4, 3)
plt.plot(X, z3)

plt.subplot(3, 4, 4)
plt.plot(X, z4)

plt.subplot(3, 4, 5)
plt.plot(X, z5)

plt.subplot(3, 4, 6)
plt.plot(X, z6)

plt.subplot(3, 4, 7)
plt.plot(X, z7)

plt.subplot(3, 4, 8)
plt.plot(X, z8)

plt.subplot(3, 4, 9)
plt.plot(X, z9)

plt.subplot(3, 4, 10)
plt.plot(X, z10)

plt.subplot(3, 4, 11)
plt.plot(X, z11)

In [ ]:
grid = hcipy.make_pupil_grid(1000)
R, th = grid.as_('polar').coords
X, Y = grid.coords

z1 = np.ones(X.shape) 
z2 = np.sqrt(24/5)*X
z3 = np.sqrt(24/5)*Y
z4 = np.sqrt(720/43)*(R**2 - 5/12)
z5 = np.sqrt(60/7)*(R**2)*np.cos(2*th)
z6 = np.sqrt(60/7)*(R**2)*np.sin(2*th)
z7 = np.sqrt(84000/737)*(R**2 - 14/25)*R*np.cos(th)
z8 = np.sqrt(84000/737)*(R**2 - 14/25)*R*np.sin(th)
z9 = np.sqrt(1517040/4987)*(R**4 - 257/301*(R**2) + 737/6020)
z10 = np.sqrt(1120/103)*(R**3)*(np.cos(3*th))
z11 = np.sqrt(160/9)*(R**3)*(np.cos(3*th))

hcipy.imshow_field(z2, grid)
plt.colorbar()

### 1/2 factor discrepancy

In [ ]:
tel2.sm.flatten()
tel2.sm.actuators[0] = 1
mask = tel2.sm.surface

hcipy.imshow_field(mask, cmap='jet')
plt.colorbar()

In [ ]:
tel2.sm.flatten()

tel2.sm.actuators[10] = 1e-9

rms = np.sqrt(np.mean((tel2.sm.surface[np.where(mask!= 0)])**2)) * 1e9

plt.figure()
hcipy.imshow_field(tel2.sm.surface*1e9, cmap='jet')
plt.xlabel(f"RMS for one segment: {rms:.3f} nm")
cbar = plt.colorbar()
cbar.set_label("in nm", loc='center')

In [ ]:
60220*z9 = np.sqrt(1517040/4987)*(6020*R**4 - 6020* 257/301*(R**2) + 737)

In [ ]:
737*3/(np.sqrt(1072205))

In [ ]:
np.sqrt(1517040/4987) *  737/6020

In [ ]:
local_zernike_basis = hcipy.mode_basis.make_zernike_basis(n_zernikes,
                                                          tel2.segment_circumscribed_diameter,
                                                          tel2.pupil_grid.shifted(-tel2.seg_pos[0]),
                                                          starting_mode=1, radial_cutoff=False)

In [ ]:
hcipy.imshow_field(local_zernike_basis[5])
plt.colorbar()

In [ ]:
len(local_zernike_basis)

### Compare LS designs from Bry 

In [ ]:
b_apodizer = fits.getdata(os.path.join(analysis_path, '2-Hex_robust_0px_N1024.fits'))
b_lyot_stop = fits.getdata(os.path.join(analysis_path, 'LS_LUVex_02-Hex_ID0000_OD0982_no_struts_gy_ovsamp4_N1024.fits'))
b_aperture = fits.getdata(os.path.join(analysis_path, 'TelAp_LUVex_02-Hex_gy_ovsamp04_N1024.fits'))

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.title("From SCDA simualor")
hcipy.imshow_field(tel2.apodizer, mask=tel2.lyotstop, cmap='Greys_r')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("From Bry's fits file")
plt.imshow(b_apodizer * b_lyot_stop,  cmap ='Greys_r' )
plt.colorbar()

plt.savefig(os.path.join(analysis_path, 'ls_comparison.png'))

In [ ]:
tel2.sm.flatten()
scda_aperture = np.array((tel2.aperture).shaped)

plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
plt.title("From SCDA aperture")
plt.imshow(scda_aperture, cmap ='Greys_r' )
plt.colorbar()

plt.subplot(1, 3, 2)
plt.title("From Bry's fits file")
plt.imshow(b_aperture,  cmap ='Greys_r' )
plt.colorbar()

plt.subplot(1, 3, 3)
plt.title("Differences between them")
plt.imshow(b_aperture-scda_aperture,  cmap ='Greys_r' )
plt.colorbar()
plt.tight_layout()
plt.savefig(os.path.join(analysis_path, 'mirror_comparison.png'))